In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

/Users/zander/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/zander/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
round(scipy.stats.norm.ppf(1 - 0.003 / 2.), 4)

2.9677

In [29]:
ns = 104
an = 11037

pv = ns/an/(1 - ns/an)
pv/(1-pv)

0.009603841536614645

In [3]:
round((189./11034.) - (104./11037.), 4)

0.0077

In [4]:
def proportions_confint_diff_ind(p1, p2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
       
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ (11034.) + p2 * (1 - p2)/ (11037.))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ (11034.) + p2 * (1 - p2)/ (11037.))
    
    return (left_boundary, right_boundary)

In [5]:
X = proportions_confint_diff_ind((189./11034.), (104./11037.))

In [6]:
round(X[1], 4)

0.0107

In [9]:
Z1 = np.random.randint(low = 1, high = 2, size=104)
Z1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [10]:
Z2 = np.random.randint(low = 0, high = 1, size=(11037 - 104))
Z2

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
Z = np.concatenate((Z1, Z2), axis=0)

In [12]:
W1 = np.random.randint(low = 1, high = 2, size=189)
W2 = np.random.randint(low = 0, high = 1, size=(11034 - 189))
W = np.concatenate((W1, W2), axis=0)

In [13]:
len(W[W==1])

189

In [15]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [16]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [17]:
def get_odds(NN):
    N1 = NN[NN==1]
    odds = float(len(N1)) / float(len(NN)) / (1 - float(len(N1)) / float(len(NN)))
    return odds

In [30]:
np.random.seed(0)

placebo_odds = list(map(get_odds, get_bootstrap_samples(W, 1000)))
aspirin_odds = list(map(get_odds, get_bootstrap_samples(Z, 1000)))

print ("95% confidence interval for the infarct odds with aspirin:",  stat_intervals(aspirin_odds, 0.05))
print ("95% confidence interval for the infarct odds with placebo:",  stat_intervals(placebo_odds, 0.05))

95% confidence interval for the infarct odds with aspirin: [0.00776114 0.01126993]
95% confidence interval for the infarct odds with placebo: [0.01480732 0.0198748 ]


In [31]:
odds_ratio = list(map(lambda x: x[1] / x[0], zip(aspirin_odds, placebo_odds)))
odds_ratio1 = list(map(lambda x: x[1] / x[0], zip(aspirin_odds, placebo_odds)))

In [32]:
print("95% confidence interval for the ratio between odds",  stat_intervals(odds_ratio, 0.05))

95% confidence interval for the ratio between odds [1.46286276 2.35093673]


In [33]:
round(stat_intervals(odds_ratio, 0.05)[0], 4)

1.4629

In [37]:
round(get_odds(Z), 4)

0.0095